In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sagemaker
import boto3
import matplotlib.pyplot as plt 
pd.set_option('display.max_columns',None) # To display all columns
pd.set_option('display.max_rows',None) # To display all columns
client = boto3.client('s3')
import feather
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from functools import reduce
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
#pip install feather-format

## LOADING DATAFRAMES

In [3]:
df1 = pd.read_feather('FinalData.ftr')
df2 = pd.read_feather('Dataframe80.ftr')
dfu = pd.read_feather('s3://haseebdata/Dataset/Users.ftr')
dfq = pd.read_feather('s3://haseebdata/Dataset/dfq.ftr')
Repeat=pd.read_feather('RepeatedQuestion.ftr')

## DF1 = MAIN COMPLETE DATAFRAME

In [4]:
df1.head()

,index,timestamp,solving_id,question_id,user_answer,elapsed_time,user_iD,correct_answer,bundle_id,explanation_id,deployed_at,correct
0,0,1565096190868,1,5012,b,38.0,1,c,3544,3544,2019-09-17 02:43:28.616,0.0
1,1,1565096221062,2,4706,c,24.0,1,c,3238,3238,2019-10-30 05:34:30.723,1.0
2,2,1565096293432,3,4366,b,68.0,1,b,2898,2898,2019-10-30 05:37:51.451,1.0
3,3,1565096339668,4,4829,a,42.0,1,c,3361,3361,2019-06-24 09:24:51.347,0.0
4,4,1565096401774,5,6528,b,59.0,1,d,5060,5060,2019-10-17 03:03:15.009,0.0


## DF2 = Percent 80 & above users


In [5]:
df2.head()

,user_iD,question_id,user_answer,correct_answer,correct,elapsed_time,solving_id,explanation_id,bundle_id,timestamp,deployed_at
0,50,3654,d,d,1.0,47.0,1,2186,2186,1540815498477,2019-09-17 02:49:05.091
1,50,898,c,c,1.0,17.0,2,898,898,1540815563576,2017-12-29 15:34:47.831
2,50,3633,c,b,0.0,53.0,3,2165,2165,1540815619061,2019-05-17 12:10:59.583
3,50,5805,a,d,0.0,41.0,4,4337,4337,1540815663799,2018-02-20 05:45:24.616
4,50,5680,d,c,0.0,25.0,5,4212,4212,1540815691595,2019-09-16 12:05:05.072


## DF3 = Deleted Duplicates Questions , Remaining Dataframe

In [6]:
ix = df2[df2.correct.eq(1)].groupby(['user_iD', 'question_id'])['elapsed_time'].idxmax()
df3= df2.loc[ix]
df3[0:5]

,user_iD,question_id,user_answer,correct_answer,correct,elapsed_time,solving_id,explanation_id,bundle_id,timestamp,deployed_at
1818,50,3,b,b,1.0,23.0,1729,3,3,1560337981670,2019-05-17 11:40:22.784
2481,50,4,b,b,1.0,29.0,2109,4,4,1560769502979,2019-05-17 11:40:25.357
2149,50,5,c,c,1.0,24.0,1883,5,5,1560513025097,2019-05-17 11:40:28.439
1838,50,7,d,d,1.0,23.0,1747,7,7,1560339686648,2019-05-17 11:40:33.992
8355,50,29,d,d,1.0,19.0,3905,29,29,1571227991581,2019-09-17 02:45:04.653


In [ ]:
df3.reset_index()

In [ ]:
df3.to_feather('df.ftr')

In [ ]:
df3['user_iD'].nunique()

In [ ]:
df3['bundle_id'].nunique()

In [ ]:
df3['question_id'].nunique()

In [ ]:
df3['question_id'].describe()

In [ ]:
df_i = df3.groupby(['user_iD', 'question_id']).size().reset_index(name='counts')
df_i.head()

In [ ]:
df_i['counts'].nunique()

## DF4 = Users with the total # of questions

In [ ]:
df_total_questions_attempted = df3.groupby(['user_iD'], as_index=False , sort=False)['question_id'].count()
df_total_questions_attempted = df_total_questions_attempted.rename(columns={'question_id': 'Unique Attempted'})

df_correct = df3[df3['correct']==1].groupby(['user_iD'], as_index=False, sort=False)['question_id'].count()

df_correct = df_correct.rename(columns={'question_id': 'Corrected'})

#df_incorrect = df3[df3['correct']==0].groupby(['user_iD'], as_index=False, sort=False)['question_id'].count()

#df_incorrect = df_incorrect.rename(columns={'question_id': 'Incorrected'})

dfs = [df_total_questions_attempted, df_correct]
df4 = reduce(lambda left,right: pd.merge(left,right,on='user_iD'), dfs)

df4['Percentage'] = (df4['Corrected'] / df4['Unique Attempted']) *100

#df_final['elapsed_time'] = df2['elapsed_time'] * 1000

#df2['elapsed_time']= df2.elapsed_time.div(1000)   #DIVDING THE ELAPSED TIME by 1000 to further check the mean elapsed time by each user
#Elapsed_time= df2.groupby('user_iD' , sort=False)['elapsed_time'].median()

#dfs = [df_final, Elapsed_time]
#df4 = reduce(lambda left,right: pd.merge(left,right,on='user_iD'), dfs)

In [ ]:
df4.head()

In [ ]:
df4['user_iD'].nunique()

In [ ]:
df4['user_iD'].nunique()